In [1]:
from pathlib import Path
import os
import numpy as np
import unicodedata
import torch
import string

### Adquirindo os dados

[Download dos dados](https://download.pytorch.org/tutorial/data.zip)

In [2]:
PATH_DATA = Path('data/names')
os.path.exists(PATH_DATA)

True

In [3]:
def remove_accents(text):
    # O decode ascii no final, serve para tirar a letra b (significa binary) do inicio dos nomes (b'nome')
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore')#.decode('ascii')

In [4]:
names = []
labels = []

# Adquire todos os caminhos de arquivos com extensão txt
for path in PATH_DATA.glob('*.txt'):
    # Abre cada arquivo txt no formato utf 8
    with open(path, 'r', encoding='utf8') as f:
        # Para cada linha no arquivo
        for line in f:
            # Remove os acentos do nome e a quebra de linha (\n) e adiciona no array
            names.append(remove_accents(line.replace('\n', '')))
            # Adiciona apenas o nome do arquivo, sem o caminho anterior e extensão no array
            labels.append(path.stem)

print('nomes: ', len(names))
print('nacionalidades: ', len(labels))

nomes:  20074
nacionalidades:  20074


### Convertendo os dados para tensor

Neste passo, será realizado a conversão dos dados e os rótulos para uma representação numérica (tensorial) que estará pronta para entrar na rede neural.

In [5]:
# Cria uma lista que não permite valores duplicados
labels_unique = sorted(list(set(labels)))

def label_to_tensor(label):
    # Baseado na label enviada pelo paramêtro, retorna o índice no array de labels
    position = labels_unique.index(label)
    # Retorna um tensor baseado na posição do índice
    return torch.LongTensor([position])

In [6]:
labels_unique

['Arabic',
 'Chinese',
 'Czech',
 'Dutch',
 'English',
 'French',
 'German',
 'Greek',
 'Irish',
 'Italian',
 'Japanese',
 'Korean',
 'Polish',
 'Portuguese',
 'Russian',
 'Scottish',
 'Spanish',
 'Vietnamese']

In [7]:
labels_tensor = [label_to_tensor(label) for label in labels]
labels_tensor[-1]

tensor([17])

#### Convertendo os rótulos para tensor

Representação One-Hot encoding para codificar os nomes

In [8]:
dictionary = string.ascii_letters

# Temos estas letras, porém, em outras linguas o espaço, hífen e apóstrofes fazem parte dos nomes de alguns países
dictionary

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [9]:
# Iremos adicionar estes caracteres no nosso dicionário
dictionary = dictionary + " -'"
dictionary

"abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ -'"

Para cada letra do nome, terá uma representação One Hot Encoding do tamanho do dicionário, ou seja, as colunas serão as letras do dicionário e os valores serão preenchidos com zeros (false) e apenas será 1 na posição referente do dicionário.

Exemplo: Caso o nome for "ana", o índice 0 será true (que representa o valor a) na primeira letra, na segunda letra (n) o índice referente a letra n será true, e assim por diante.

In [10]:
def name_to_tensor(name):
    # Será criado um array bi dimensional onde: 
    # A primeira dimensão representa cada letra do nome
    # A segunda dimensão representa todas as letras do dicionário
    name_tensor = torch.zeros(len(name), len(dictionary))
    for idx, letter in enumerate(name.decode('utf-8')):
        # Adquire o índice da letra (idx) no name_tensor
        # Para o índice da letra no dicionário (segunda dimensão) define como 1 (true)
        index = dictionary.find(letter) # Busca a letra no dicionário, caso não encontrar, retorna -1
        name_tensor[idx][index] = 1
        
    return name_tensor

In [ ]:
names_tensor = [name_to_tensor(name) for name in names]
names_tensor[-1] # Exibe o tensor do último nome

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.]])

#### Amostrando conjunto balaceado